In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-

In [3]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding

In [4]:
openai.api_key = ""
# or
os.environ['OPENAI_API_KEY'] = ""

In [5]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/AI-Republic-PH/AIR_AI_Engineering_Course_2024/refs/heads/main/Datasets/IMDB_Dataset.csv', nrows=20)

In [6]:
dataframed.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
dataframed['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [9]:
System_Prompt = """Your role is to analyze and classify the sentiment of movie reviews, categorizing each as either "Positive" or "Negative." You must provide concise, one-word classifications. Below are detailed guidelines to ensure accurate assessment:

1. Classification Scope
Objective: Focus on identifying the overall sentiment conveyed by the reviewer. This sentiment can be determined by examining the language used, the tone of the review, and the degree of satisfaction or dissatisfaction expressed.
Clarity: Do not interpret or summarize details beyond the sentiment classification; your response should only indicate "Positive" or "Negative."
2. Binary Response Only
Strictly Binary: The response must be limited to either "Positive" or "Negative." No other terms, explanations, or neutrality options are allowed.
Consistency: Ensure the classification remains clear and unambiguous for every review.
3. Criteria for Positive Sentiment
Expressed Satisfaction: If the reviewer indicates that they enjoyed the movie, appreciated its aspects (acting, direction, plot, etc.), or were generally pleased with their experience, classify the review as "Positive."
Engagement or Enthusiasm: Signs of excitement, recommendation to others, or expressions like "hooked," "thrilling," "amazing," or similar terms signal a positive sentiment.
Tolerance of Negative Elements: If a reviewer mentions flaws but concludes with an overall positive feeling or appreciation, lean towards classifying it as "Positive."
4. Criteria for Negative Sentiment
Expressed Dissatisfaction: If the reviewer expresses disappointment, criticism, or dissatisfaction with the movie’s elements (acting, plot, visuals, etc.), categorize it as "Negative."
Disengagement or Regret: If the review indicates that the reviewer was bored, confused, or felt their time was wasted, mark it as "Negative."
Dominant Negative Tone: Even if there are some positive mentions, if the overall tone and conclusion focus on faults or lack of enjoyment, classify it as "Negative."
5. Handling Neutral or Mixed Sentiments
Dominant Sentiment Rule: If a review appears neutral or contains mixed sentiments, determine the most dominant aspect of the sentiment. For instance, if a review mentions both strengths and weaknesses, but the reviewer leans more toward criticism or praise, use that as your classification basis.
Avoid Neutrality: Always choose either "Positive" or "Negative" based on the dominant tone, even if the review seems balanced.
Examples for Clarification:
Example 1 (Positive): "This movie had its flaws, but I couldn’t help but love the chemistry between the characters. I highly recommend it!"
Classification: Positive (Despite mentioning flaws, the overall tone is enthusiastic and appreciative.)
Example 2 (Negative): "The film had impressive visuals, but the storyline was all over the place and made no sense. I wouldn’t watch it again."
Classification: Negative (Though visuals are praised, the dominant sentiment is critical, focusing on the poor storyline.)
By following these criteria, you ensure a consistent and clear classification of movie reviews into either "Positive" or "Negative."""

In [10]:
struct = [{'role' : 'system', 'content' : System_Prompt}]

user_message = dataframed['review'][0]
print("User:", user_message)
struct.append({"role": "user", "content": user_message})
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages = struct)
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})
print("Assistant:", response)

User: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to t

In [11]:
dataframed['Openai_Sentiment'] = ''

In [13]:
# Assuming dataframed is already defined
for x in range(len(dataframed)):
    struct = [{'role': 'system', 'content': System_Prompt}]
    user_message = dataframed.loc[x, 'review']
    struct.append({"role": "user", "content": user_message})
    chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages=struct)
    response = chat.choices[0].message.content
    struct.append({"role": "assistant", "content": response})

    # Use .loc[] for assignment
    dataframed.loc[x, 'Openai_Sentiment'] = response

/tmp/ipykernel_30/3244958445.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframed['Openai_Sentiment'][x] = response
/tmp/ipykernel_30/3244958445.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You 

In [14]:
dataframed.head(20)

,review,sentiment,Openai_Sentiment
0,One of the other reviewers has mentioned that ...,positive,Positive
1,A wonderful little production. <br /><br />The...,positive,Positive
2,I thought this was a wonderful way to spend ti...,positive,Positive
3,Basically there's a family where a little boy ...,negative,Negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Positive
5,"Probably my all-time favorite movie, a story o...",positive,Positive
6,I sure would like to see a resurrection of a u...,positive,Positive
7,"This show was an amazing, fresh & innovative i...",negative,Negative
8,Encouraged by the positive comments about this...,negative,Negative
9,If you like original gut wrenching laughter yo...,positive,Positive
